In [1]:
import pandas as pd
from tqdm import tqdm_notebook
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
import gensim
import logging
import nltk.data
import urllib.request
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer
nltk.download('punkt')
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\FED\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html

In [3]:
# считываем датасеты с нормированными описаниями тендеров

train_data_desc_stemmed = pd.read_csv('../data/intermid/train_data_desc_stemmed.csv')
test_data_desc_stemmed = pd.read_csv('../data/intermid/test_data_desc_stemmed.csv')

In [4]:
# соединяем датасеты

data = pd.concat([train_data_desc_stemmed, test_data_desc_stemmed], ignore_index=True)
data

,pn_lot_id,text_description_tender_stemmed
0,7031618,услуга проведение финансовый аудит
1,7808247,приобретение приз проведение конкурс вопрос из...
2,7009496,приобретение приз проведение мероприятие избир...
3,5938735,продление лицензия битрикс управление сайт веб...
4,9327348,приобретение компьютерный техника машина вычис...
...,...,...
862994,5373821,право заключение договор выполнение строительн...
862995,8396902,право заключение договор выполнение строительн...
862996,7569089,запрос ценовый предложение электронный форма п...
862997,2403905,право заключение договор выполнение проектный ...


In [5]:
data.shape

(862999, 2)

In [6]:
# Чтобы обучить модель, нам нужно связать тег/номер с каждым документом корпуса обучения.
# В нашем случае тег - это просто отсчитываемый от нуля номер строки.

In [7]:
# функция для чтения корпуса

def read_corpus(corpus, tokens_only=False):
    for i, line in enumerate(corpus):
        tokens = gensim.utils.simple_preprocess(line)
        if tokens_only:
            yield tokens
        else:
            # For training data, add tags
            yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

In [8]:
%%time
train_corpus = list(read_corpus(data.text_description_tender_stemmed))

Wall time: 45 s


In [9]:
train_corpus[:5]

[TaggedDocument(words=['услуга', 'проведение', 'финансовый', 'аудит'], tags=[0]),
 TaggedDocument(words=['приобретение', 'приз', 'проведение', 'конкурс', 'вопрос', 'избирательный', 'процесс', 'гражданин', 'ограниченный', 'физический', 'возможность', 'переферийной', 'оборудование', 'компьютер', 'телефон', 'головной', 'наушник', 'комбинированный', 'устройство', 'состоять', 'микрофон'], tags=[1]),
 TaggedDocument(words=['приобретение', 'приз', 'проведение', 'мероприятие', 'избиратель', 'акустический', 'колонка', 'портативный', 'аккумулятор', 'внешний', 'накопитель', 'данные', 'устройство', 'запоминать'], tags=[2]),
 TaggedDocument(words=['продление', 'лицензия', 'битрикс', 'управление', 'сайт', 'веб', 'кластер', 'услуга', 'оптовый', 'торговля', 'программный', 'обеспечение'], tags=[3]),
 TaggedDocument(words=['приобретение', 'компьютерный', 'техника', 'машина', 'вычислительный', 'электронный', 'цифровой', 'прочий', 'содержать', 'корпус', 'следующий', 'устройство', 'автоматический', 'обрабо

In [10]:
# Cоздадим экземпляр модели Doc2Vec с векторным размером с 100 измерениями
# и выполним итерацию по обучающей совокупности 100 раз.
# Устанавливаем минимальное количество слов равным 2, 
# чтобы отбрасывать слова с очень небольшим количеством вхождений.

In [11]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=100, min_count=2, epochs=100)

In [12]:
# создаем словарь
model.build_vocab(train_corpus)

2021-09-25 02:09:06,302 : INFO : collecting all words and their counts
2021-09-25 02:09:06,302 : INFO : PROGRESS: at example #0, processed 0 words (0/s), 0 word types, 0 tags
2021-09-25 02:09:06,396 : INFO : PROGRESS: at example #10000, processed 271176 words (2961070/s), 16585 word types, 10000 tags
2021-09-25 02:09:06,465 : INFO : PROGRESS: at example #20000, processed 496971 words (2883304/s), 26755 word types, 20000 tags
2021-09-25 02:09:06,550 : INFO : PROGRESS: at example #30000, processed 733854 words (2801369/s), 31307 word types, 30000 tags
2021-09-25 02:09:06,619 : INFO : PROGRESS: at example #40000, processed 939423 words (2881244/s), 34268 word types, 40000 tags
2021-09-25 02:09:06,719 : INFO : PROGRESS: at example #50000, processed 1175590 words (2624620/s), 39160 word types, 50000 tags
2021-09-25 02:09:06,797 : INFO : PROGRESS: at example #60000, processed 1423614 words (3225136/s), 42611 word types, 60000 tags
2021-09-25 02:09:06,882 : INFO : PROGRESS: at example #70000,

2021-09-25 02:09:11,864 : INFO : PROGRESS: at example #630000, processed 14793810 words (2839058/s), 132146 word types, 630000 tags
2021-09-25 02:09:11,948 : INFO : PROGRESS: at example #640000, processed 15037504 words (2896966/s), 132764 word types, 640000 tags
2021-09-25 02:09:12,033 : INFO : PROGRESS: at example #650000, processed 15267991 words (2978699/s), 133558 word types, 650000 tags
2021-09-25 02:09:12,095 : INFO : PROGRESS: at example #660000, processed 15446249 words (2471172/s), 134982 word types, 660000 tags
2021-09-25 02:09:12,180 : INFO : PROGRESS: at example #670000, processed 15643406 words (2744195/s), 136369 word types, 670000 tags
2021-09-25 02:09:12,249 : INFO : PROGRESS: at example #680000, processed 15835695 words (2462027/s), 137441 word types, 680000 tags
2021-09-25 02:09:12,334 : INFO : PROGRESS: at example #690000, processed 16037317 words (2754534/s), 138525 word types, 690000 tags
2021-09-25 02:09:12,396 : INFO : PROGRESS: at example #700000, processed 162

In [15]:
# %%time
# # тренируем модель на корпусе
# model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

2021-09-24 23:34:41,383 : INFO : training model with 3 workers on 89109 vocabulary and 100 features, using sg=0 hs=0 sample=0.001 negative=5 window=5
2021-09-24 23:34:42,441 : INFO : EPOCH 1 - PROGRESS: at 1.20% examples, 262971 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:34:43,457 : INFO : EPOCH 1 - PROGRESS: at 2.46% examples, 250595 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:34:44,460 : INFO : EPOCH 1 - PROGRESS: at 3.68% examples, 247717 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:34:45,478 : INFO : EPOCH 1 - PROGRESS: at 4.92% examples, 234903 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:34:46,549 : INFO : EPOCH 1 - PROGRESS: at 6.22% examples, 239170 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:34:47,550 : INFO : EPOCH 1 - PROGRESS: at 7.41% examples, 241662 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:34:48,570 : INFO : EPOCH 1 - PROGRESS: at 8.69% examples, 238390 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:34:49,588 : INFO : EPOCH 1 - PROGRESS: at 9.90% exam

2021-09-24 23:35:56,738 : INFO : EPOCH 1 - PROGRESS: at 91.09% examples, 231432 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:35:57,747 : INFO : EPOCH 1 - PROGRESS: at 92.33% examples, 232044 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:35:58,818 : INFO : EPOCH 1 - PROGRESS: at 93.57% examples, 231724 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:35:59,874 : INFO : EPOCH 1 - PROGRESS: at 94.88% examples, 232043 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:36:00,904 : INFO : EPOCH 1 - PROGRESS: at 96.10% examples, 232167 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:36:01,960 : INFO : EPOCH 1 - PROGRESS: at 97.39% examples, 232349 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:36:02,971 : INFO : EPOCH 1 - PROGRESS: at 98.65% examples, 232544 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:36:03,994 : INFO : EPOCH 1 - PROGRESS: at 99.86% examples, 232558 words/s, in_qsize 4, out_qsize 0
2021-09-24 23:36:04,074 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-

2021-09-24 23:37:08,038 : INFO : EPOCH 2 - PROGRESS: at 77.71% examples, 237554 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:37:09,056 : INFO : EPOCH 2 - PROGRESS: at 78.91% examples, 236791 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:37:10,059 : INFO : EPOCH 2 - PROGRESS: at 80.12% examples, 236229 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:37:11,061 : INFO : EPOCH 2 - PROGRESS: at 81.38% examples, 235811 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:37:12,117 : INFO : EPOCH 2 - PROGRESS: at 82.59% examples, 235113 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:37:13,135 : INFO : EPOCH 2 - PROGRESS: at 83.86% examples, 234667 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:37:14,238 : INFO : EPOCH 2 - PROGRESS: at 85.10% examples, 233949 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:37:15,241 : INFO : EPOCH 2 - PROGRESS: at 86.36% examples, 233728 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:37:16,290 : INFO : EPOCH 2 - PROGRESS: at 87.60% examples, 233080 words/s, in_qsiz

2021-09-24 23:38:20,305 : INFO : EPOCH 3 - PROGRESS: at 65.28% examples, 238789 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:38:21,323 : INFO : EPOCH 3 - PROGRESS: at 66.49% examples, 238582 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:38:22,379 : INFO : EPOCH 3 - PROGRESS: at 67.78% examples, 238871 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:38:23,443 : INFO : EPOCH 3 - PROGRESS: at 69.04% examples, 238862 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:38:24,477 : INFO : EPOCH 3 - PROGRESS: at 70.38% examples, 238726 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:38:25,587 : INFO : EPOCH 3 - PROGRESS: at 71.62% examples, 239473 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:38:26,589 : INFO : EPOCH 3 - PROGRESS: at 72.89% examples, 238676 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:38:27,639 : INFO : EPOCH 3 - PROGRESS: at 74.20% examples, 238861 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:38:28,695 : INFO : EPOCH 3 - PROGRESS: at 75.44% examples, 238567 words/s, in_qsiz

2021-09-24 23:39:32,081 : INFO : EPOCH 4 - PROGRESS: at 52.77% examples, 237577 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:39:33,078 : INFO : EPOCH 4 - PROGRESS: at 54.02% examples, 237447 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:39:34,132 : INFO : EPOCH 4 - PROGRESS: at 55.26% examples, 238065 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:39:35,151 : INFO : EPOCH 4 - PROGRESS: at 56.52% examples, 237593 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:39:36,154 : INFO : EPOCH 4 - PROGRESS: at 57.76% examples, 237371 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:39:37,210 : INFO : EPOCH 4 - PROGRESS: at 58.98% examples, 237369 words/s, in_qsize 5, out_qsize 0
2021-09-24 23:39:38,213 : INFO : EPOCH 4 - PROGRESS: at 60.25% examples, 237979 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:39:39,215 : INFO : EPOCH 4 - PROGRESS: at 61.44% examples, 238352 words/s, in_qsize 6, out_qsize 0
2021-09-24 23:39:40,250 : INFO : EPOCH 4 - PROGRESS: at 62.69% examples, 238102 words/s, in_qsiz

...

2021-09-25 01:48:00,823 : INFO : EPOCH 98 - PROGRESS: at 15.02% examples, 244835 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:48:01,888 : INFO : EPOCH 98 - PROGRESS: at 16.28% examples, 244806 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:48:02,906 : INFO : EPOCH 98 - PROGRESS: at 17.61% examples, 241692 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:48:03,931 : INFO : EPOCH 98 - PROGRESS: at 18.83% examples, 240214 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:48:04,934 : INFO : EPOCH 98 - PROGRESS: at 20.08% examples, 241815 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:48:05,937 : INFO : EPOCH 98 - PROGRESS: at 21.30% examples, 242950 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:48:07,001 : INFO : EPOCH 98 - PROGRESS: at 22.60% examples, 245381 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:48:08,073 : INFO : EPOCH 98 - PROGRESS: at 23.92% examples, 243925 words/s, in_qsize 5, out_qsize 0
2021-09-25 01:48:09,076 : INFO : EPOCH 98 - PROGRESS: at 25.16% examples, 243240 words/s

2021-09-25 01:49:12,766 : INFO : EPOCH 99 - PROGRESS: at 2.42% examples, 248844 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:49:13,769 : INFO : EPOCH 99 - PROGRESS: at 3.63% examples, 246578 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:49:14,772 : INFO : EPOCH 99 - PROGRESS: at 4.92% examples, 237632 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:49:15,853 : INFO : EPOCH 99 - PROGRESS: at 6.18% examples, 238801 words/s, in_qsize 5, out_qsize 0
2021-09-25 01:49:16,862 : INFO : EPOCH 99 - PROGRESS: at 7.41% examples, 242824 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:49:17,925 : INFO : EPOCH 99 - PROGRESS: at 8.69% examples, 237552 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:49:18,946 : INFO : EPOCH 99 - PROGRESS: at 9.96% examples, 243957 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:49:19,986 : INFO : EPOCH 99 - PROGRESS: at 11.16% examples, 249601 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:49:21,036 : INFO : EPOCH 99 - PROGRESS: at 12.44% examples, 247362 words/s, in_qs

2021-09-25 01:50:27,287 : INFO : EPOCH 99 - PROGRESS: at 92.75% examples, 232446 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:50:28,321 : INFO : EPOCH 99 - PROGRESS: at 94.02% examples, 232337 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:50:29,393 : INFO : EPOCH 99 - PROGRESS: at 95.27% examples, 232605 words/s, in_qsize 5, out_qsize 0
2021-09-25 01:50:30,465 : INFO : EPOCH 99 - PROGRESS: at 96.56% examples, 232734 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:50:31,467 : INFO : EPOCH 99 - PROGRESS: at 97.79% examples, 232802 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:50:32,470 : INFO : EPOCH 99 - PROGRESS: at 99.04% examples, 233000 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:50:33,250 : INFO : worker thread finished; awaiting finish of 2 more threads
2021-09-25 01:50:33,250 : INFO : worker thread finished; awaiting finish of 1 more threads
2021-09-25 01:50:33,272 : INFO : worker thread finished; awaiting finish of 0 more threads
2021-09-25 01:50:33,272 : INFO : EPOCH - 99

2021-09-25 01:51:38,270 : INFO : EPOCH 100 - PROGRESS: at 79.20% examples, 237437 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:51:39,331 : INFO : EPOCH 100 - PROGRESS: at 80.46% examples, 236808 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:51:40,350 : INFO : EPOCH 100 - PROGRESS: at 81.72% examples, 236298 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:51:41,399 : INFO : EPOCH 100 - PROGRESS: at 83.00% examples, 235759 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:51:42,440 : INFO : EPOCH 100 - PROGRESS: at 84.25% examples, 235246 words/s, in_qsize 5, out_qsize 0
2021-09-25 01:51:43,458 : INFO : EPOCH 100 - PROGRESS: at 85.49% examples, 234810 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:51:44,461 : INFO : EPOCH 100 - PROGRESS: at 86.71% examples, 234433 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:51:45,464 : INFO : EPOCH 100 - PROGRESS: at 87.95% examples, 233931 words/s, in_qsize 6, out_qsize 0
2021-09-25 01:51:46,514 : INFO : EPOCH 100 - PROGRESS: at 89.21% examples, 23347

Wall time: 2h 17min 14s


In [14]:
# Сохраняем модель

model_path = '../models/doc2vec_c2_e100.model'
# print("Saving model...")
# model.save(model_path)

In [15]:
# загружаем сохраненную модель

model = gensim.models.doc2vec.Doc2Vec.load(model_path)

2021-09-25 02:13:11,621 : INFO : loading Doc2Vec object from ../models/doc2vec_c2_e100.model
2021-09-25 02:13:12,518 : INFO : loading vocabulary recursively from ../models/doc2vec_c2_e100.model.vocabulary.* with mmap=None
2021-09-25 02:13:12,522 : INFO : loading trainables recursively from ../models/doc2vec_c2_e100.model.trainables.* with mmap=None
2021-09-25 02:13:12,522 : INFO : loading wv recursively from ../models/doc2vec_c2_e100.model.wv.* with mmap=None
2021-09-25 02:13:12,522 : INFO : loading docvecs recursively from ../models/doc2vec_c2_e100.model.docvecs.* with mmap=None
2021-09-25 02:13:12,522 : INFO : loading vectors_docs from ../models/doc2vec_c2_e100.model.docvecs.vectors_docs.npy with mmap=None
2021-09-25 02:13:13,650 : INFO : loaded ../models/doc2vec_c2_e100.model


In [19]:
# получаем векторы описаний используя обученную модель, передав список слов в функцию model.infer_vector.

vectors = []
for doc_id in tqdm(range(len(train_corpus))):
    vectors.append(model.infer_vector(train_corpus[doc_id].words))

100%|███████████████████████████████████████████████████████████████████████| 862999/862999 [1:44:50<00:00, 137.20it/s]


In [20]:
# vectors

In [21]:
# создаем датасет из идентификаторов и векторов и сохраняем в csv файл для дальнейшего использования

vectors_df = pd.DataFrame(data.pn_lot_id).join(pd.DataFrame(vectors))
vectors_df

,pn_lot_id,0,1,2,3,4,5,6,7,8,...,90,91,92,93,94,95,96,97,98,99
0,7031618,0.072818,-0.189699,-0.120791,-0.198305,0.207617,0.092488,-0.011035,-0.211811,0.050383,...,0.037440,0.155822,0.013580,0.100133,-0.124113,0.105949,-0.179645,-0.144118,0.052846,0.074452
1,7808247,0.085868,0.856505,0.774590,-0.160336,0.449811,0.898541,0.072058,0.684850,-0.389740,...,0.157330,-0.312800,-0.244867,-0.112409,0.024903,0.376228,-1.378329,-0.255605,0.097167,-0.149609
2,7009496,1.093472,0.002100,-0.584886,-1.037389,0.115569,0.471769,-0.444472,0.059995,0.478849,...,-0.567645,-0.364759,-0.646891,-0.625459,-0.285858,-0.940639,-1.020377,-0.478274,0.712764,0.018372
3,5938735,-0.272242,-0.151343,-0.237722,-0.292642,0.176220,0.415424,0.570904,0.350875,0.521188,...,0.094355,0.634943,-0.467935,0.215553,-0.293477,-0.043282,-0.429736,-0.499643,0.425119,0.226768
4,9327348,-0.028881,-0.006010,0.113354,-0.598772,0.293816,0.194816,0.001027,-0.206447,0.438054,...,0.045352,0.262884,0.060446,0.638687,-0.137093,0.237627,-0.237250,-0.204133,0.679995,-0.014523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
862994,5373821,0.550107,-0.430535,-0.571353,0.449493,0.391876,1.098007,1.506350,0.377086,0.232366,...,1.219116,0.527213,-0.190420,1.241750,-0.612602,-0.639951,-0.265407,-1.126294,0.051408,0.065189
862995,8396902,0.254296,-0.329352,-0.588136,-0.130033,0.414901,1.038211,1.002123,0.290325,0.408057,...,1.080652,0.594704,-0.758843,0.823547,-0.402396,-0.885107,-0.147024,-0.843586,-0.037301,-0.044134
862996,7569089,-1.321565,0.139032,1.059917,2.090054,0.634147,0.681203,3.150911,-0.411310,1.309038,...,-0.014100,-1.220033,0.260207,-0.443016,-0.475499,0.942912,-0.526614,-0.167450,0.040999,-0.034304
862997,2403905,0.053751,-0.258513,-0.475770,0.518367,0.054118,0.284792,0.165399,-0.165183,-0.088283,...,-0.430948,0.467167,-0.388497,-0.058068,-0.539636,0.418210,-1.038308,-1.213415,-0.147827,0.539468


In [22]:
vectors_df.to_csv('../data/intermid/doc2Vec_vectors.csv', index=False, encoding='utf-8')